# Text Context Analysis
This notebook deals with the text context analysis part of the proposed solution.

In [1]:
#imported necessary packages
from pydub import AudioSegment
import pygame
from transformers import pipeline
import logging
import ffmpeg

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#defined function that plays an audio file

def play_audio(file_path):
    pygame.init()
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(file_path)#loads the audio file
        pygame.mixer.music.play()#plays the audio file
        while pygame.mixer.music.get_busy():#waits until the audio playback is finished before continuing with the rest of the program
            pygame.time.Clock().tick(10)
    except pygame.error as e:
        print("Error occurred while playing audio:", e) #prints an error message should the program fails to be executed
    pygame.quit()

In [3]:
#defined function that can play audio through the speakers (or headphones)

def play(input_file):
    #called function that plays the newly converted wav audio file
    play_audio(input_file)
    

In [5]:
#Using pipeline, created a transcriber that when called, converts an audio file and transcribes it into text
transcriber = pipeline(task="automatic-speech-recognition", model="openai/whisper-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
#Configures logging to write logs to a file named transcript.log
logging.basicConfig(filename='transcript.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#defined a function that would utilize the transcriber
def transcribe(input_file):
    print("Transcribing. Please wait...")
    try:
        transcript = transcriber(input_file)#calls the transcriber
        logging.info("Transcript: %s", transcript)  #Logs the transcript
        print("Transcript:", transcript)  #Prints the transcript
    except Exception as e:#should transcriber fail, the messages below will be displayed and the user will be prompted to enter transcript manually
        logging.error("Error occurred: %s", e)  # Log the error
        print("Error occurred:", e)  # Print the error message

    return transcript


In [9]:
#This approach uses the badmatr11x/distilroberta-base-offensive-hateful-speech-text-multiclassification model instead of the
##cardiffnlp/twitter-roberta-base-sentiment-latest model
# #defined function that determines whether a customer is angry or not

# def is_caller_angry(input_file):
#     #called transcribe function
#     transcript = transcribe(input_file)
    
#     #Used pipeline for sentiment analysis using distilroberta fine-tuned on offensive/hateful dataset since our focus is on angry customers
#     from transformers import pipeline

#     classify_sentiment = pipeline("text-classification", 
#                                   model="badmatr11x/distilroberta-base-offensive-hateful-speech-text-multiclassification")

#     try:
#         sentiment = classify_sentiment(transcript['text'])
#         score = sentiment[0]['score'] #assigns score to score variable

#     except:
#         sentiment = 'negative'

#     #extracts the label from the sentiment
#     try:
#         sentiment_label = sentiment[0]['label']
#     except:
#         sentiment_label = sentiment

#     if sentiment_label != 'NEITHER':#because other labels are indicative of anger
#         #if this condition is met, then statement should be considered to be coming from someone who is angry

#         print("User is angry.")       

#     else:
#         print("User is calm.")


#     return sentiment_label
    

In [9]:
#defined function that determines whether a customer is angry or not

def is_caller_angry(input_file):
    #called transcribe function
    transcript = transcribe(input_file)
    
    #Used pipeline for sentiment analysis 
    from transformers import pipeline
    
    classify_sentiment = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

    try:
        sentiment = classify_sentiment(transcript['text'])
        score = sentiment[0]['score'] #assigns score to score variable

    except:
        sentiment = 'negative'

    #extracts the label from the sentiment
    try:
        sentiment_label = sentiment[0]['label']
    except:
        sentiment_label = sentiment

    if sentiment_label == 'negative':
        #if this condition is met, then statement should be considered to be coming from someone who is angry

        print("User is angry.")       

    else:
        print("User is calm.")


    return sentiment_label
    